In [7]:
##  COPY TO POTZY PACKAGE, TAKE THAT AS THE MAIN ONE
import requests
import string
import pandas as pd

SET_META_URL_MAIN = 'https://www.set.or.th/dat/eod/listedcompany/static/listedCompanies_en_US.xls'
OUTPUT_FILE = '../data/meta/thai_stocks.csv'
stock_table = pd.read_html(SET_META_URL_MAIN)[0]
columns = stock_table.iloc[1]
stock_table = stock_table.iloc[2:]
stock_table.columns = [x.lower() for x in columns]
stock_table.set_index('symbol',inplace=True)
stock_table.index.name='ticker'

############################### OLD STUFF ###########################################
# SET_META_URL_PREFIX = 'https://www.set.or.th/set/commonslookup.do?language=th&country=TH&prefix='
# prefix_list = ["NUMBER"]+list(string.ascii_uppercase[:27])
# stock_table = pd.DataFrame()
# for prefix in prefix_list:
#     stock_table = stock_table.append(pd.read_html(SET_META_URL_PREFIX+prefix)[0],ignore_index=True)    
# stock_table.columns = ['ticker','name','market']
# stock_table.set_index('ticker',inplace=True)
# ############################### OLD STUFF ###########################################


#print result
print('Print thai stock metadata headers ...')
print(stock_table.head())
print('Saving result to ',OUTPUT_FILE)

#Save result
stock_table.to_csv(OUTPUT_FILE)

Print thai stock metadata headers ...
                                       company market  \
ticker                                                  
2S             2S METAL PUBLIC COMPANY LIMITED    mai   
A       AREEYA PROPERTY PUBLIC COMPANY LIMITED    SET   
AAV       ASIA AVIATION PUBLIC COMPANY LIMITED    SET   
ABICO    ABICO HOLDINGS PUBLIC COMPANY LIMITED    mai   
ABM        ASIA BIOMASS PUBLIC COMPANY LIMITED    mai   

                       industry                      sector  \
ticker                                                        
2S                   Industrial                           -   
A       Property & Construction        Property Development   
AAV                    Services  Transportation & Logistics   
ABICO      Agro & Food Industry                           -   
ABM                   Resources                           -   

                                                  address zip code  \
ticker                                            